# 시청률 데이터 전처리

- 시청률 데이터를 이용하여 방송시간 때의 평균 시청률을 column에 추가한다.

In [101]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from math import ceil
from dateutil.parser import parse

In [102]:
# 시청률 데이터
df_rating = pd.read_excel('./data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_시청률 데이터.xlsx',header=1,index_col='시간대')

In [103]:
# 양쪽 맨 끝에 있는 평균값 제거하기
df_rating = df_rating.iloc[:1440]
df_rating = df_rating.iloc[:,:-1]

컬럼은 날짜, 인덱스는 시간대이다.   

이 둘을 합쳐서 '2019-01-01 09:00:00'의 형태와 시청률을 매칭시킨다.

In [111]:
# 데이터 index와 column을 합친다
date = []
rating = []
for day in df_rating.columns:
    for time in df_rating.index:
        date.append('{} {}'.format(day,time))
        rating.append(df_rating.loc[time,day])
        
df_time = pd.DataFrame({'date':date,'rating':rating})
df_time

,date,rating
0,2019-01-01 02:00,0.0
1,2019-01-01 02:01,0.0
2,2019-01-01 02:02,0.0
3,2019-01-01 02:03,0.0
4,2019-01-01 02:04,0.0
...,...,...
525595,2019-12-31 01:55,0.0
525596,2019-12-31 01:56,0.0
525597,2019-12-31 01:57,0.0
525598,2019-12-31 01:58,0.0


In [112]:
# 빠른 실행을 위하여 0값 드랍
df_time = df_time[df_time.rating !=0]
df_time

,date,rating
9,2019-01-01 02:09,0.014
100,2019-01-01 03:40,0.014
101,2019-01-01 03:41,0.011
102,2019-01-01 03:42,0.011
103,2019-01-01 03:43,0.011
...,...,...
525497,2019-12-31 00:17,0.024
525498,2019-12-31 00:18,0.050
525499,2019-12-31 00:19,0.050
525500,2019-12-31 00:20,0.024


In [113]:
# 홈쇼핑 데이터와 시간 기준을 같게 만들고 str을 datetime으로 변경
def ys(date):
    if date[-5:]<'02:00':
        return datetime.strptime(date,'%Y-%m-%d %H:%M') + timedelta(days=1)
    else:
        return datetime.strptime(date,'%Y-%m-%d %H:%M')
df_time.date = df_time.date.apply(ys)

C:\Users\Playdata\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [114]:
df_time.sort_values('date')

,date,rating
9,2019-01-01 02:09:00,0.014
100,2019-01-01 03:40:00,0.014
101,2019-01-01 03:41:00,0.011
102,2019-01-01 03:42:00,0.011
103,2019-01-01 03:43:00,0.011
...,...,...
525497,2020-01-01 00:17:00,0.024
525498,2020-01-01 00:18:00,0.050
525499,2020-01-01 00:19:00,0.050
525500,2020-01-01 00:20:00,0.024


In [115]:
# date를 인덱스로 사용
df_time.set_index('date',inplace=True)

In [81]:
# 홈쇼핑 데이터
df_ns = pd.read_csv('data/df_data_revised.csv',index_col='Unnamed: 0')
df_ns

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,2
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,2
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,2
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,2
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,2
...,...,...,...,...,...,...,...,...,...,...
38299,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1
38300,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,2
38301,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,2
38302,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,2


In [90]:
# date 타입을 datetime으로 바꾸기
df_ns.date = df_ns.date.map(parse)

In [97]:
# 편성표에 맞추어 시청률 평균값 구하기
def rating_TV(row,df_time):
    st = str(row.date)
    fi = str(row.date + timedelta(minutes=ceil(row.exp_mins))) # 시간을 올림하였다.
    return( df_time[st:fi].sum())/ceil(row.exp_mins) # 0값을 날렸기에 더하고 기간으로 나눈다.
df_ns['rating'] = df_ns.apply(rating_TV,df_time=df_time,axis=1)

In [98]:
df_ns

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,rating
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,2,0.0000
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,2,0.0000
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,2,0.0000
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,2,0.0000
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,2,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
38299,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,0.0206
38300,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,2,0.0000
38301,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,2,0.0000
38302,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,2,0.0000


In [100]:
df_time.sort_index()

,rating
date,
2019-01-02 00:34:00,0.012
2019-01-02 00:35:00,0.012
2019-01-02 00:36:00,0.012
2019-01-02 00:37:00,0.012
2019-01-02 00:38:00,0.012
...,...
2019-12-31 23:55:00,0.024
2019-12-31 23:56:00,0.024
2019-12-31 23:57:00,0.024


In [95]:
# 저장
df_ns.to_csv